In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

# Read in training and test data

In [3]:
df_train = pd.read_csv("./Data/train.csv")
df_test = pd.read_csv("./Data/test.csv")

# Response variable

In [4]:
y_train = df_train[['SalePrice']]
y_train = y_train.fillna(y_train.mean()).values.ravel()
y_train = np.log1p(y_train)

# Set up covariate matrix

In [5]:
X_train = df_train.drop(['Id', 'SalePrice'], axis=1)
X_test = df_test.drop(['Id'], axis=1)
X_all = pd.concat([X_train, X_test], axis=0)

In [6]:
X_all_cat = X_all.select_dtypes(include=['object'])
X_all_num = X_all.select_dtypes(exclude=['object'])

In [7]:
#create dummy variables for categorical variables
X_all_cat = pd.get_dummies(X_all_cat)

In [8]:
#impute missing values
X_all_cat = X_all_cat.fillna(X_all_cat.mean())
X_all_num = X_all_num.fillna(X_all_num.mean())